In [1]:
from utils import *
import numpy as np
import pandas as pd

pd.set_option("display.notebook_repr_html", False) 

Zrob kolumne od badges w ktorej bedzie string z odznakami(Participation Badges, awarded once) (bez spacji), połącz to po user Id z Posts_questions i Posts_answers (w sumie z dwóch ostatnich możesz zostawić tylko UserId i CreationDate) czyli:

1. Funkcja do Badges, która wyjmie userId i kolumne ze stringiem z odznakami(Participation Badges, awarded once) (bez spacji)
2. Funckja do Posts_questions i Posts_answers - wyjmie UserId i CreationDate.
3. Funckja ktora stworzy słownik - godzina i ilosc postow
4. dalej z tego mozna okreslic:
    + jaka godzina jest popularna/najmniej popularna wsrod danej religii ogolnie - wykres, 
    + jaka godzina najczestsza/najrzadsza wsrod uzytkownikow z dana odznaką -wykres,
    + to samo globalnie albo określić jakieś przedziały w stylu 'rano', 'noc', 'pora obiadowa'

In [2]:
buddyzm = Religion('../buddyzm')
islam = Religion('../islam')
judaism = Religion('../judaism')

W tym punkcie naszym celem było zbadanie zależności między czasem tworzenia posta lub dodania odpowiedzi, religią, a odznakami użytkownika. W tym celu potrzebowaliśmy wyodrębnić ramkę danych stworzoną z UserId i wszystkimi odznakami danego użytkownika.
Zajmuje się tym funkcja `participation_badges`.

In [43]:
    def participation_badges(religia):
        """funkcja, która tworzy ramkę danych ParticipationBadges, zawierająca kolumnę UserId i kolumnę UserBadges,
        której wiersze to stringi ParticipationBadges danego użytkownika (przyznawanych tylko raz).
        ParticipationBadges są określone na podstawie
            https://meta.stackexchange.com/questions/67397/what-are-the-badges-i-can-earn-on-each-site-and-what-are-the-exact-criteriafor#188731"""

        UserBadges = ['Autobiographer', 'Commentator', 'Pundit', 'Enthusiast', 'Fanatic', 'Mortarboard', 'Epic',
                      'Legendary', 'Precognitive', 'Beta', 'Quorum', 'Convention', 'Talkative', 'Outspoken']
        ParticipationBadges = religia.Badges[['UserId','Name']].copy()

        # tworzę ramkę posiadającą jedynie kolumne UserId z unikatowymi UserId
        ParticipationBadges_final = ParticipationBadges[['UserId']].drop_duplicates()

        # tworze nowa kolumne, w której znajda sie wypisane ParticipationBadges oddzielone'-', danego użytkownika
        # (dla każdego użytkownika - po UserId- jest tworzona lista nazw przyznanych mu Badges, jeśli nazwa należy do UserBadges to jest dopisywana do stringa)
        ParticipationBadges_final['UserBadges'] = ["-".join(
            [s for s in ParticipationBadges.loc[ParticipationBadges['UserId'] == t, ['Name']].Name.tolist() if
             s in UserBadges]) for t in ParticipationBadges_final['UserId']]

        # usuwam wiersze użytkowników, którzy nie posiadają żadnego ParticipationBadges - w tej kolumnie zostąły wpisane puste stringi
        nan_value = float("NaN")
        ParticipationBadges_final.replace("", nan_value, inplace=True)
        ParticipationBadges_final.dropna(subset=['UserBadges'], inplace=True)

        # resetuje indexy
        ParticipationBadges_final.reset_index(drop =True, inplace = True)
        return ParticipationBadges_final

In [4]:
# bPBadges =participation_badges(buddyzm)
# bPBadges
jPBadges =participation_badges(judaism)
iPBadges =participation_badges(islam)


In [5]:
#tu mozesz zrobic fora ktory bedzie tworzyl te regexy dla godzin i wgl zrob z tego funkcje
jPosts = judaism.Posts.copy()
jPosts_questions = jPosts[jPosts.PostTypeId=='1'].copy()
jPosts_questions= jPosts_questions[jPosts_questions['CreationDate'].str.match("^[0-9]{4}-[0-9]{2}-[0-9]{2}T20")==True]
len(jPosts_questions) #tyle bedzie pytan utworzonych o danej godzinie

2103

In [6]:
jPosts_answers =jPosts[jPosts.PostTypeId=='2'].copy()
jPosts_answers[jPosts_answers['CreationDate'].str.match("^[0-9]{4}-[0-9]{2}-[0-9]{2}T20")==True]


           Id PostTypeId AcceptedAnswerId             CreationDate Score  \
37         43          2              NaN  2009-12-10T20:20:25.247    33   
134       153          2              NaN  2010-01-01T20:18:42.733     5   
192       218          2              NaN  2010-01-07T20:49:45.847     1   
210       238          2              NaN  2010-01-14T20:46:07.333     4   
211       240          2              NaN  2010-01-14T20:58:42.053     7   
...       ...        ...              ...                      ...   ...   
87498  120484          2              NaN  2021-02-23T20:39:12.620     4   
87540  120531          2              NaN  2021-02-24T20:20:28.847     2   
87576  120573          2              NaN  2021-02-25T20:37:34.973     0   
87577  120574          2              NaN  2021-02-25T20:56:55.060     2   
87608  120606          2              NaN  2021-02-27T20:29:11.487     2   

      ViewCount                                               Body  \
37          NaN  

In [44]:
%%time
    # bPBadges =participation_badges(buddyzm)
# bPBadges
jPBadges =participation_badges(judaism)
iPBadges =participation_badges(islam)
print(jPBadges)

     UserId                                         UserBadges
0         2  Autobiographer-Commentator-Precognitive-Quorum...
1         3  Commentator-Mortarboard-Quorum-Precognitive-Co...
2         5  Commentator-Precognitive-Quorum-Talkative-Mort...
3         7                                        Commentator
4         9                    Commentator-Precognitive-Quorum
...     ...                                                ...
4180  24983                                     Autobiographer
4181  24987                                     Autobiographer
4182  24993                                     Autobiographer
4183  24994                                     Autobiographer
4184  24996                                     Autobiographer

[4185 rows x 2 columns]
Wall time: 1min 40s


In [92]:
    def participation_badges(religia):# ZMIENIONE
        """funkcja, która tworzy ramkę danych ParticipationBadges, zawierająca kolumnę UserId i kolumnę UserBadges,
        której wiersze to stringi ParticipationBadges danego użytkownika (przyznawanych tylko raz).
        ParticipationBadges są określone na podstawie
            https://meta.stackexchange.com/questions/67397/what-are-the-badges-i-can-earn-on-each-site-and-what-are-the-exact-criteriafor#188731"""
        UserBadges = ['Autobiographer', 'Commentator', 'Pundit', 'Enthusiast', 'Fanatic', 'Mortarboard', 'Epic',
                      'Legendary', 'Precognitive', 'Beta', 'Quorum', 'Convention', 'Talkative', 'Outspoken']
        ParticipationBadges = religia.Badges[['UserId','Name']].copy()

        # tworzę ramkę posiadającą jedynie kolumne UserId z unikatowymi UserId

        # tworze nowa kolumne, w której znajda sie wypisane ParticipationBadges oddzielone'-', danego użytkownika
        # (dla każdego użytkownika - po UserId- jest tworzona lista nazw przyznanych mu Badges, jeśli nazwa należy do UserBadges to jest dopisywana do stringa)
        ParticipationBadges=ParticipationBadges[ParticipationBadges['Name'].isin(UserBadges)]
        ParticipationBadges['Name']+='-'
        ParticipationBadges=ParticipationBadges.groupby('UserId').sum().reset_index()
        ParticipationBadges['UserId'] = ParticipationBadges['UserId'].astype(int) 
        ParticipationBadges=ParticipationBadges.sort_values('UserId')
        
        
        # resetuje indexy
        ParticipationBadges.reset_index()
        return ParticipationBadges

In [93]:
%%time
    # bPBadges =participation_badges(buddyzm)
# bPBadges
jPBadges =participation_badges(judaism)
iPBadges =participation_badges(islam)
print(jPBadges)

      UserId                                               Name
1979       2  Autobiographer-Commentator-Precognitive-Quorum...
2927       3  Commentator-Mortarboard-Quorum-Precognitive-Co...
3225       5  Commentator-Precognitive-Quorum-Talkative-Mort...
3580       7                                       Commentator-
3933       9                   Commentator-Precognitive-Quorum-
...      ...                                                ...
2794   24983                                    Autobiographer-
2795   24987                                    Autobiographer-
2796   24993                                    Autobiographer-
2797   24994                                    Autobiographer-
2798   24996                                    Autobiographer-

[4185 rows x 2 columns]
Wall time: 789 ms
